In [ ]:
# a setting for my cluster; ignore it
import os
os.environ['CUDA_MPS_PIPE_DIRECTORY'] = "/tmp/nvidia-mps"
os.environ['CUDA_MPS_LOG_DIRECTORY'] = "/tmp/nvidia-log"

In [ ]:
%%time
from nngp import *

In [ ]:
random_state = 4623457
ndim = 10
N_train, N_test, N_pool = 200, 200, 1000
layers = [128,64,32]

np.random.seed(random_state)

In [ ]:
from scipy.optimize import rosen
def ans(x):
    return rosen(x.T)[:,None]
X_train = np.random.random((N_train, ndim))
y_train = ans(X_train)

X_pool = np.random.random((N_pool, ndim))
y_pool = ans(X_pool)

X_test = np.random.random((N_test, ndim))
y_test = ans(X_test)

print('shapes:', X_train.shape, y_train.shape)
print('shapes:', X_test.shape, y_test.shape)
print('shapes:', X_pool.shape, y_pool.shape)

In [ ]:
tf.reset_default_graph()
nn = NN(
    ndim = ndim,
    random_state = random_state,
    layers = layers
)

In [ ]:
try:
    sess.close()
except:
    pass
# a setting for my cluster; ignore it
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

# global init
init = tf.global_variables_initializer()
saver = tf.train.Saver()
sess = tf.Session(config=config)
sess.run(init)

In [ ]:
nn.train(
    session = sess,
    X_train = X_train,
    y_train = y_train,
    X_test = X_test,
    y_test = y_test,
    X_val = X_test,
    y_val = y_test,
)

rmses = [np.sqrt(mse(nn.predict(sess, data = X_test), y_test))]

for al_iteration in range(1, 10):
    print(f'[{al_iteration}] BEFORE:')
    print('shapes:', X_train.shape, y_train.shape)
    print('shapes:', X_test.shape, y_test.shape)
    print('shapes:', X_pool.shape, y_pool.shape)
    
    gpue = nn.get_nngp_UE(sess, X_train, y_train, X_pool, y_pool)
    
    X_train, y_train, X_pool, y_pool = \
        update_learning_sets(X_train,
                             y_train,
                             X_pool,
                             y_pool,
                             gpue,
                             sample_size = 100)
    
    print(f'[{al_iteration}] AFTER:')
    print('shapes:', X_train.shape, y_train.shape)
    print('shapes:', X_test.shape, y_test.shape)
    print('shapes:', X_pool.shape, y_pool.shape)
    
    nn.train(
        session = sess,
        X_train = X_train,
        y_train = y_train,
        X_test = X_test,
        y_test = y_test,
        X_val = X_test,
        y_val = y_test,
    )
    rmses.append(np.sqrt(mse(nn.predict(sess, data = X_test),
            y_test)))

In [ ]:
rmses

In [ ]:
nn.predict(sess,
           data = X_test[:3])

In [ ]:
y_test[:3]

In [ ]:
gpue = nn.get_nngp_UE(sess, X_train, y_train, X_pool, y_pool)
mcd_ue = nn.get_mcd_UE(sess, X_pool)
gpue, mcd_ue

In [ ]:
print('shapes:', X_train.shape, y_train.shape)
print('shapes:', X_test.shape, y_test.shape)
print('shapes:', X_pool.shape, y_pool.shape)

In [ ]:
X_train, y_train, X_pool, y_pool = \
    update_learning_sets(X_train,
                         y_train,
                         X_pool,
                         y_pool,
                         gpue,
                         sample_size = 10)

In [ ]:
print('shapes:', X_train.shape, y_train.shape)
print('shapes:', X_test.shape, y_test.shape)
print('shapes:', X_pool.shape, y_pool.shape)